In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils import FeatureExtractor2, Loader, f_scorer

Using TensorFlow backend.


In [2]:
train = Loader().load_dataset('./data/train/train_stances.csv', './data/train/train_bodies.csv')

In [3]:
test = Loader().load_dataset('./data/test/test_stances.csv', './data/test/test_bodies.csv')

In [4]:
params = {
    'headline_len':64,
    'body_len': 800,
    'max_words':150000
}

In [5]:
fe = FeatureExtractor2(headline_len=params['headline_len'], body_len=params['body_len'], max_words=params['max_words']).fit(train, test)

In [6]:
train_headlines,train_bodies, train_stances = fe.transform(train, labels=True)

In [7]:
train_headlines.shape, train_bodies.shape, train_stances.shape

((49972, 64), (49972, 800), (49972,))

In [8]:
from keras.layers import Embedding, Input, CuDNNGRU, Bidirectional, Concatenate, Dense, Dropout
from keras.models import Model
from keras.regularizers import l2

In [19]:
def get_model():
    headline_input = Input(shape=(params['headline_len'],))
    headline_embedding = Embedding(input_dim=params['max_words'], output_dim=64)(headline_input)
    headline_encoder = Bidirectional(CuDNNGRU(70))(headline_embedding)
    
    body_input = Input(shape=(params['body_len'],))
    body_embedding = Embedding(input_dim=params['max_words'], output_dim=100)(body_input)
    body_encoder = Bidirectional(CuDNNGRU(100))(body_embedding)
    
    context = Concatenate()([headline_encoder, body_encoder])
    dense = Dense(64,activation='relu')(context)
    dropout = Dropout(0.3)(dense)
    output = Dense(4, activation='softmax')(dropout)
    
    model = Model(inputs=[headline_input, body_input], outputs=[output])
    model.summary()
    return model

In [20]:
model = get_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 800)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 64, 64)       9600000     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 800, 100)     15000000    input_4[0][0]                    
__________________________________________________________________________________________________
bidirectio

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [12]:
history = model.fit([train_headlines, train_bodies], train_stances, batch_size=64, epochs=12)

Train on 39977 samples, validate on 9995 samples
Epoch 1/12
39977/39977 [==============================] - 114s 3ms/step - loss: 0.9607 - acc: 0.7675 - val_loss: 0.5849 - val_acc: 0.7990
Epoch 2/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.5829 - acc: 0.8095 - val_loss: 0.5553 - val_acc: 0.8124
Epoch 3/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.5243 - acc: 0.8292 - val_loss: 0.4877 - val_acc: 0.8336
Epoch 4/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.4586 - acc: 0.8488 - val_loss: 0.4448 - val_acc: 0.8500
Epoch 5/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.4219 - acc: 0.8583 - val_loss: 0.4200 - val_acc: 0.8591
Epoch 6/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.3953 - acc: 0.8647 - val_loss: 0.4097 - val_acc: 0.8579
Epoch 7/12
39977/39977 [==============================] - 113s 3ms/step - loss: 0.3792 - acc: 0.8678 - val_loss: 0.3964 - v

In [13]:
test_headlines, test_bodies, test_stances = fe.transform(test, labels=True)

In [14]:
import numpy as np

In [15]:
preds = model.predict([test_headlines, test_bodies])
preds = np.argmax(preds, axis=1)

In [16]:
print(f_scorer(y_pred = preds, y_true=test_stances))

4729.25


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

In [18]:
print(classification_report(y_pred = preds, y_true=test_stances))
print(confusion_matrix(y_pred = preds, y_true=test_stances))

             precision    recall  f1-score   support

          0       0.10      0.25      0.14      1903
          1       0.20      0.29      0.24      4464
          2       0.73      0.57      0.64     18349
          3       0.00      0.00      0.00       697

avg / total       0.57      0.48      0.52     25413

[[  476   366  1061     0]
 [  725  1283  2456     0]
 [ 3325  4518 10506     0]
 [  153   131   413     0]]


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
